In [1]:
import pandas as pd
import os
import awswrangler as wr
env = os.environ.get('Environment')
print(f"Environment: {env!r}")


Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

In [3]:
%%athena_to_df --database f"{env}-luna-pharma-prepared-data" --out df_test 
select ar.group_id, ar.categorie, ar.code_article, ar.libelle,
    jh.job_name, jh.sw_no_lot, jh.browse_description, jh.sw_job_type, jh.job_status, 
    sa.id_numeric, sa.status as sample_status, sa.sample_type, sa.recd_date,
    sa.id_text, jh.date_completed,jh.date_authorised,jh.sw_decision,jh.sw_decision_date,jh.sw_decision_id
    
from sample sa 

join job_header jh on jh.job_name = sa.job_name
join sw_article ar on ar.code_article = jh.sw_article

where ar.group_id like 'ARA%'
and jh.job_status = 'A'
and jh.sw_lot_validation = 'F'
and jh.sw_no_lot not like '%FOR%'
and from_iso8601_timestamp(jh.date_authorised) BETWEEN current_date - INTERVAL '7' day AND current_date;


In [4]:
df_test.head(5)

,group_id,categorie,code_article,libelle,job_name,sw_no_lot,browse_description,sw_job_type,job_status,id_numeric,sample_status,sample_type,recd_date,id_text,date_completed,date_authorised,sw_decision,sw_decision_date,sw_decision_id
0,ARAMON,PF,ARA_EPUEVM,EAU PURIFIEE,ARA_EPUEVM01-23,01-23,EAU PURIFIEE,PF,A,2582186,A,PF,2022-12-22T10:08:58,Q_ARA_EPUEVM01-23_892,2023-03-07T11:00:41,2023-03-07T11:02:02,,2022-12-22T10:08:09,
1,ARAMON,PF,ARA_EPUEVM,EAU PURIFIEE,ARA_EPUEVM01-23,01-23,EAU PURIFIEE,PF,A,2582187,A,PF,2022-12-22T10:09:03,Q_ARA_EPUEVM01-23_893,2023-03-07T11:00:41,2023-03-07T11:02:02,,2022-12-22T10:08:09,
2,ARAMON,MP,ARA_639199,AMIODARONE HCL,ARA_63919923AR002,23AR002,AMIODARONE HCL,MP,A,2587162,A,MP,2023-01-09T10:07:13,Q_ARA_63919923AR002_217,2023-03-01T11:09:19,2023-03-03T11:51:37,,2023-01-03T13:27:12,
3,ARAMON,MP,ARA_639199,AMIODARONE HCL,ARA_63919923AR002,23AR002,AMIODARONE HCL,MP,A,2587159,A,MP,2023-01-09T10:06:58,Q_ARA_63919923AR002_215,2023-03-01T11:09:19,2023-03-03T11:51:37,,2023-01-03T13:27:12,
4,ARAMON,MP,ARA_639199,AMIODARONE HCL,ARA_63919923AR002,23AR002,AMIODARONE HCL,MP,A,2587160,A,MP,2023-01-09T10:07:09,Q_ARA_63919923AR002_216,2023-03-01T11:09:19,2023-03-03T11:51:37,,2023-01-03T13:27:12,


In [5]:
database_name = f'prod-dmdg-workspace-public-aramon-smartqc-approval-20230113150753781'

In [6]:
table_name = f'{env}_ARA_luna_smartqc_Approval'

In [7]:
path = f's3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-approval-20230113150753781'

In [8]:
workspace_prefix =f'public-aramon-smartqc-approval-20230113150753781'

In [9]:
wr.s3.to_parquet(
    df=df_test,
    path=path,
    database=database_name,
    dataset=True,
    table=table_name,
    mode='overwrite',
    index=False,
    compression=None,
)

{'paths': ['s3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-approval-20230113150753781/eca23d1e86fe457c90a2cfa0cd346911.parquet'],
 'partitions_values': {}}

In [10]:
df_test.shape

(188, 19)